In [13]:
from collections import *
import heapq

data = None

with open('test.txt', 'r') as file:
    data = file.readlines()
    data = [d.strip() for d in data]
#print(data)

In [14]:
m,n = len(data),len(data[0])
#print(m,n)
def getLetter(x,y):
    if 0 <= x < n and 0 <= y < m:
        return data[y][x]
    return " "
diagonals = [[1,1],[2,2],[3,3]]
orthogonals =[[0,1],[0,2],[0,3]]

ds = [[],[],[],[]]
for da,db in diagonals:
    for i,(dx,dy) in enumerate([[1,1],[-1,-1],[1,-1],[-1,1]]):
        ds[i].append([da*dx,db*dy])
do = [[],[],[],[]]
for da,db in orthogonals:
    #[a,b] : a = x or y,b = up or down 
    for i,(dx,dy) in enumerate([[1,1],[-1,-1],[1,-1],[-1,1]]):
        value = db*dy
        if dx == 1:
            do[i].append([0,value])
        else:
            do[i].append([value,0])
directions = ds + do
#print(directions)

In [15]:
sum = 0
for y in range(0,m):
    for x in range(0,n):
        if getLetter(x,y) != "X":
            continue
        for d in directions:
            s = ""
            for dy,dx in d:
                s += getLetter(x + dx,y + dy)
            #print(s)
            if s == "MAS":
                sum += 1
print(sum)
                

18


In [16]:
sum = 0
for y in range(0,m):
    for x in range(0,n):
        tl = getLetter(x,y) 
        if not (tl == "M" or tl == "S"):
            continue
        tl = getLetter(x,y)
        tr = getLetter(x+2,y)
        bl = getLetter(x,y+2)
        br = getLetter(x+2,y+2)
        middle = getLetter(x+1,y+1)
        counter = Counter([tl,tr,bl,br])
        if "M" not in counter or "S" not in counter:
            continue
        if counter["M"] != 2 or counter["S"] != 2:
            continue
        if middle != "A":
            continue
        if tl == br or bl == tr:
            continue
        sum += 1
print(sum)

9


In [18]:
import copy
import sys
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Now you can import the file
from viz import VideoGenerator
sum = 0
decay = 32
fps = 24

stategrid =[[0 for x in row] for row in data]
#5x5 fonts
X =[
    [1,0,0,0,1],
    [0,1,0,1,0],
    [0,0,1,0,0],
    [0,1,0,1,0],
    [1,0,0,0,1]
]
M =[
    [0,1,0,1,0],
    [0,1,0,1,0],
    [1,0,1,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1]
]
A = [
    [0,0,1,0,0],
    [0,1,0,1,0],
    [0,1,1,1,0],
    [1,0,0,0,1],
    [1,0,0,0,1]
]
S = [
    [0,1,1,1,1],
    [1,0,0,0,0],
    [0,1,1,1,0],
    [0,0,0,0,1],
    [1,1,1,1,0]
]

#offset,letter,rgb
letters = {
    'X':(0,X,(0,103,56)),
    'M':(1,M,(68,74,46)),
    'A':(2,A,(112,55,38)),
    'S':(3,S,(102, 75, 0))
}
lettersize = 5

GREY_CODE = 0
WHITE_CODE = 250
colourDict={
    GREY_CODE:(100,100,100),
    WHITE_CODE:(255,255,255)       
} #white

def getInbetweenWhite(colour,currdecay):
    mult = (1/decay)*currdecay
    tupdiff = [(255-colour[i])*mult for i in range(3)] # diff between colour and white
    tup = tuple(int(colour[i]+tupdiff[i]) for i in range(3))
    return tup

def decayIdx(i,offset):
    return i*4+offset+1
    
for c in 'XMAS':
    offset,_,colour = letters[c]
    for i in range(decay+1):
        idx = decayIdx(i,offset)
        colourDict[idx] = getInbetweenWhite(colour,i)
print(colourDict)
def concatArray(l,r):
    return [left+right for left,right in zip(l,r)]
def printFrame(frame):
    #print(frame)
    for row in frame:
        print(''.join(["." if x==0 else "#" for x in row]))
        

def getFrame(stategrid,cx,cy,directions):
    for y in range(m):
        for x in range(n):
            stategrid[y][x] = max(stategrid[y][x] -1,0)
    for direction in directions:
        for x,y in direction:
            stategrid[y][x] = decay

    filledinrow = [0 for x in range((lettersize+1)*n + 1)]
    frame = [filledinrow[:]]

    for y,(staterow,letterrow) in enumerate(zip(stategrid,data)):
        fillcolumn = [[0] for i in range(lettersize)]
        scanline = copy.deepcopy(fillcolumn)
        for x,(state,letter) in enumerate(zip(staterow,letterrow)):
            offset,graphic,colour = letters[letter]
            decayidx = decayIdx(state,offset)
            colouredGraphic = [[cell*decayidx for cell in row] for row in graphic]
            scanline = concatArray(scanline,colouredGraphic)
            scanline = concatArray(scanline,fillcolumn)
        #print('scanline',scanline)
        frame += scanline
        frame.append(filledinrow[:])

    
    #cursor
    big_y = cy*(lettersize+1)+1
    big_x = cx*(lettersize+1)+1
    for y in range(big_y,big_y+lettersize):
        frame[y][big_x] = WHITE_CODE
        frame[y][big_x+1] = WHITE_CODE
        frame[y][big_x+2] = WHITE_CODE
        frame[y][big_x+3] = WHITE_CODE
        frame[y][big_x+4] = WHITE_CODE
        
    
    return frame,stategrid
        

Video = VideoGenerator("aoc2024day4",colourDict)
for i in range(fps*1): # hold for 1 secs
    frame,stategrid = getFrame(stategrid,0,0,[])
   # printFrame(frame)

    Video.draw_frame(frame)
    Video.print_frame()



for y in range(0,m):
    for x in range(0,n):
        if getLetter(x,y) != "X":
            frame,stategrid = getFrame(stategrid,x,y,[])
            Video.draw_frame(frame)
            Video.print_frame()
            continue
        xmas_directions = []
        for d in directions:
            s = ""
            current_d = [(x,y)]
            for dy,dx in d:
                s += getLetter(x + dx,y + dy)
                current_d.append((x + dx,y + dy))
            #print(s)
            if s == "MAS":
                sum += 1
                xmas_directions.append(current_d)
        frame,stategrid = getFrame(stategrid,x,y,xmas_directions)
        Video.draw_frame(frame)
        Video.print_frame()
for i in range(fps*3): # hold for 1 secs
    frame,stategrid = getFrame(stategrid,n-1,m-1,[])
    Video.draw_frame(frame)
    Video.print_frame()

Video.print_video(framerate=fps)
print(sum)





{0: (100, 100, 100), 250: (255, 255, 255), 1: (0, 103, 56), 5: (7, 107, 62), 9: (15, 112, 68), 13: (23, 117, 74), 17: (31, 122, 80), 21: (39, 126, 87), 25: (47, 131, 93), 29: (55, 136, 99), 33: (63, 141, 105), 37: (71, 145, 111), 41: (79, 150, 118), 45: (87, 155, 124), 49: (95, 160, 130), 53: (103, 164, 136), 57: (111, 169, 143), 61: (119, 174, 149), 65: (127, 179, 155), 69: (135, 183, 161), 73: (143, 188, 167), 77: (151, 193, 174), 81: (159, 198, 180), 85: (167, 202, 186), 89: (175, 207, 192), 93: (183, 212, 199), 97: (191, 217, 205), 101: (199, 221, 211), 105: (207, 226, 217), 109: (215, 231, 223), 113: (223, 236, 230), 117: (231, 240, 236), 121: (239, 245, 242), 125: (247, 250, 248), 129: (255, 255, 255), 2: (68, 74, 46), 6: (73, 79, 52), 10: (79, 85, 59), 14: (85, 90, 65), 18: (91, 96, 72), 22: (97, 102, 78), 26: (103, 107, 85), 30: (108, 113, 91), 34: (114, 119, 98), 38: (120, 124, 104), 42: (126, 130, 111), 46: (132, 136, 117), 50: (138, 141, 124), 54: (143, 147, 130), 58: (149, 

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.4.0 (GCC)
  configuration: --prefix=/home/josh/anaconda3 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1634179802525/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1634179802525/_build_env/bin/pkg-config
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 / 55.  9.

18


frame=  196 fps= 72 q=-0.0 Lsize=     740kB time=00:00:08.14 bitrate= 745.0kbits/s speed=2.99x    
video:740kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.002638%
